In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py import DFTB_methods
from libra_py import units


/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator

## 1. Calculations of the H and S for a single geometry

In [2]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-18.2.x86_64-linux/bin/dftb+"
md_file = "md.xyz"
sp_gen_file = "x1.gen"
syst_spec = "C"
scf_in_file = "dftb_in_ham1.hsd"
hs_in_file = "dftb_in_ham2.hsd"

Generate the "x1.gen" file

In [3]:
i = 0
DFTB_methods.xyz_traj2gen_sp(md_file, sp_gen_file, i, syst_spec)

Run SCF calculations and generate the charge density for a converged calculations.

The file x1.gen is used as a geometry.

Generate:

* autotest.tag
* band.out
* charges.bin
* detailed.out
* dftb_in.hsd
* dftb_pin.hsd
* eigenvec.bin
* eigenvec.out

In [4]:
os.system("cp %s dftb_in.hsd" % scf_in_file )
os.system( "%s" % EXE )

0

Lets clean up the directory a bit - move all the produced files to a separate folder

In [5]:
os.system( "rm -r calc1")
os.system( "mkdir calc1")
files = ["autotest.tag", "band.out", "charges.bin", "detailed.out",
         "dftb_in.hsd", "dftb_pin.hsd", "eigenvec.bin", "eigenvec.out"]
for it in files:
    os.system( "mv %s calc1" % it)

We need the charges.bin, so lets copy it back

Then - just generate the Hamiltonian corresponding to the converged density matrix.

In [6]:
os.system("cp calc1/charges.bin .")

os.system("cp %s dftb_in.hsd" % hs_in_file )
os.system( "%s" % EXE )

0

As before - lets move all the generated files into another directory

In [7]:
os.system( "rm -r calc2")
os.system( "mkdir calc2")
files = ["band.out", "charges.bin", "detailed.out",
         "dftb_in.hsd", "dftb_pin.hsd", "hamsqr1.dat", "oversqr.dat"]
for it in files:
    os.system( "mv %s calc2" % it)

Read the converged Hamiltonian - constructed using the converged charge density

Read the AO overlap matrix

In [8]:
Hi = DFTB_methods.get_dftb_matrices("calc2/hamsqr1.dat")
Si = DFTB_methods.get_dftb_matrices("calc2/oversqr.dat")

In [9]:
# Get the dimensions
ao_sz = Hi[0].num_of_cols
mo_sz = ao_sz

ao_act_sp = range(0, ao_sz)
mo_act_sp = range(0, mo_sz)

Extract submatrix of the orbitals of interest

Solve the eigenvalue problem and get the orbitals

In [10]:
# Extract the sub-matrix of interest
H_sub = CMATRIX(ao_sz, mo_sz)
pop_submatrix(Hi[0], H_sub, ao_act_sp, mo_act_sp)  # last element #0 = gamma-point

# Get the orbitals
Ei = CMATRIX(mo_sz, mo_sz)
MOi = CMATRIX(ao_sz, mo_sz)
solve_eigen(H_sub, Si[0], Ei, MOi, 0)     # last element #0 = gamma-point

Lets print out the orbitals and compare the what DFTB+ gives

In [11]:
MOi.show_matrix("MOs.txt")

In [12]:
f = open("MO.0.txt", "w")

for i in xrange(ao_sz):
    f.write("%i %10.8f %10.8f \n" % (i,  MOi.get(i,0).real,  MOi.get(i,0).imag) )
f.close()

One can then compare the content of the generated files (feel free to change the index of the MO) to the content of the file calc1/eigenvec.out

## 2. Calculation of the transition density matrix

Generate "x2.gen" file

In [13]:
ovlp_gen_file = "x2.gen"
ovlp_in_file = "dftb_in_overlaps.hsd"

i = 0
DFTB_methods.xyz_traj2gen_ovlp(md_file, ovlp_gen_file, i, i+1, syst_spec)

Run the calculations on the super-molecule. Only to generate the overlap matrix in the AO basis

In [14]:
os.system("cp %s dftb_in.hsd" % ovlp_in_file)
os.system( "%s" % EXE )

0

As before - organize the generated files into a separate directory

In [15]:
os.system( "rm -r calc3")
os.system( "mkdir calc3")
files = ["band.out", "detailed.out", "dftb_in.hsd", "dftb_pin.hsd", "hamsqr1.dat", "oversqr.dat"]
for it in files:
    os.system( "mv %s calc3" % it)

Read the supermatrix and split it into blocks:

In [16]:
St = DFTB_methods.get_dftb_matrices("calc3/oversqr.dat")

ao_sz2 = St[0].num_of_cols/2

block0 = range(0, ao_sz2)
block1 = range(ao_sz2, 2*ao_sz2)


S00 = CMATRIX(ao_sz2, ao_sz2)
S01 = CMATRIX(ao_sz2, ao_sz2)
S10 = CMATRIX(ao_sz2, ao_sz2)
S11 = CMATRIX(ao_sz2, ao_sz2)

pop_submatrix(St[0], S00, block0, block0) 
pop_submatrix(St[0], S01, block0, block1) 
pop_submatrix(St[0], S10, block1, block0) 
pop_submatrix(St[0], S11, block1, block1) 

In [17]:
S00.real().show_matrix("S00.txt")
S01.real().show_matrix("S01.txt")
S10.real().show_matrix("S10.txt")
S11.real().show_matrix("S11.txt")